<a href="https://colab.research.google.com/github/AliAkbarBadri/complex-networks-basics/blob/main/influence_maximization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setuping

In [ ]:
import pandas as pd
import networkx as nx
import scipy.io
import matplotlib.pyplot as plt
import collections
import numpy as np
from collections import Counter
import sys

In [ ]:
! wget "https://github.com/AliAkbarBadri/complex-networks-basics/blob/main/facebook101_princton_weighted.mat?raw=true" -O "facebook101_princton_weighted.mat"

# Reading Data

In [ ]:
mat = scipy.io.loadmat('facebook101_princton_weighted.mat')
print(mat.keys())
print(mat['A'].shape[0])
print(mat['A'])

In [ ]:
G = nx.from_scipy_sparse_matrix(mat['A'])
print(G.number_of_nodes())
print(G.number_of_edges())

In [ ]:
G.get_edge_data(6568,6595)

# Influence Maximization

# Outbreak Detection